# Notes

## Reynolds Number

$$
Re = \frac{\rho U L}{\mu} = \frac{U L}{\nu}
$$

## Prantdl Number

$$
Pr = \frac{\nu}{\alpha} = \frac{\mu}{\rho \alpha} = \frac{\mu C_p}{\kappa}
$$

$$
\alpha = \frac{\kappa}{\rho C_p}
$$
 
## Rayleigh Number

$$
Ra = \frac{g \beta \Delta T L^3}{\nu \alpha}
$$

## Normalization?

$$
\begin{aligned}
&Pr=0.71\\
&Ra=1e3\\
&\rho=1\\
&g=1\\
\nu=\frac{Pr}{\sqrt{Ra}}&\quad \mu=\rho \nu\\
&\alpha=\frac{\nu}{Pr}\\
&\Delta T = 1\\
&\kappa=1\\
&C_p=\frac{\kappa Ra}{\mu}\\
&\beta=Pr = \frac{Ra \nu \alpha}{g\Delta T L^3}
\end{aligned}
$$

In [ ]:
import os
import numpy as np
import pyvista as pv
import pandas as pd
import matplotlib.pyplot as plt
import scipy.interpolate as interpolate
import scipy.integrate as integrate

In [ ]:
ra_str = "Ra1e6N100"
file_path = "./NaturalConvectionCavityNormalizationData" + ra_str
file_list = os.listdir(file_path)
file_list.sort()
latest_file_name = file_list[-1]
latest_file_name = os.path.abspath(os.path.join(file_path, latest_file_name))
print(latest_file_name)

In [ ]:
file = latest_file_name
mesh = pv.read(file)
mesh.point_data["VelocityX"] = mesh.point_data["Velocity"][:, 0]
mesh.point_data["VelocityY"] = mesh.point_data["Velocity"][:, 1]
mesh

In [ ]:
num = 201
xs = np.linspace(0, 1, num)
ys = np.linspace(0, 1, num)
points = []
for y in ys:
    for x in xs:
        points.append([x, y, 0])
        pass
    pass
points = np.array(points)
grid = pv.PolyData(points)
grid = grid.interpolate(mesh, radius=6/(num-1))

In [ ]:
x = []
y = []
u = []
v = []
temperture = []
for i in range(0, len(grid.points)):
    x.append(grid.points[i][0])
    y.append(grid.points[i][1])
    u.append(grid.point_data["VelocityX"][i])
    v.append(grid.point_data["VelocityY"][i])
    temperture.append(grid.point_data["Temperature"][i])
    pass
x = np.array(x)
y = np.array(y)
u = np.array(u)
v = np.array(v)
temperture = np.array(temperture)

In [ ]:
plt.figure(figsize=(8, 6), facecolor="white")
plt.gca().set_aspect("equal")
ax = plt.contour(x.reshape(num, num), y.reshape(num, num), temperture.reshape(num, num), np.arange(1, 20)*0.05, colors="k", linewidths=1)
ax.clabel(fmt="%1.2f", fontsize=10)
plt.contourf(x.reshape(num, num), y.reshape(num, num), temperture.reshape(num, num), 101, cmap="rainbow")
plt.colorbar()
plt.streamplot(x.reshape(num, num), y.reshape(num, num), u.reshape(num, num), v.reshape(num, num), color="white", linewidth=1.2, density=1)
plt.xlabel("$x$", fontsize=15)
plt.ylabel("$y$", fontsize=15)
plt.title("Time=%s %s" % (str(mesh.field_data["Time"][0]), ra_str), fontsize=15)
plt.savefig("images/"+ra_str+".pdf", bbox_inches="tight", dpi=300)
plt.savefig("images/"+ra_str+".png", bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
mesh_dt = mesh.copy().delaunay_2d()
mesh_dt = mesh_dt.compute_derivative("Temperature", gradient="TemperatureGradient")
mesh_dt

In [ ]:
left_wall_y = np.linspace(0, 1, num)
left_wall_x = np.zeros_like(left_wall_y)
left_wall_z = np.zeros_like(left_wall_y)
left_wall_points = np.column_stack((left_wall_x, left_wall_y, left_wall_z))
left_wall_points = pv.PolyData(left_wall_points)
left_wall_mesh = left_wall_points.interpolate(mesh_dt, radius=3/(num-1))
left_wall_dtdx = left_wall_mesh.point_data["TemperatureGradient"][:, 0]
left_wall_dtdx = np.array(left_wall_dtdx)
left_wall_df = pd.DataFrame({"y": left_wall_mesh.points[:, 1], "dtdx": left_wall_dtdx})
left_wall_df = left_wall_df.dropna()
left_wall_df.T

In [ ]:
left_wall_min_dtdx = left_wall_df["dtdx"].values[0]
left_wall_max_dtdx = left_wall_df["dtdx"].values[-1]
left_wall_interpolate = interpolate.interp1d(left_wall_df["y"], left_wall_df["dtdx"], kind="cubic", fill_value=(left_wall_min_dtdx, left_wall_max_dtdx), bounds_error=False)
left_wall_dtdx_interpolate = left_wall_interpolate(left_wall_y)
left_wall_df_interpolate = pd.DataFrame({"y": left_wall_y, "dtdx": left_wall_dtdx_interpolate})
left_wall_df_interpolate.T

In [ ]:
nusselt_number = integrate.simps(left_wall_df_interpolate["dtdx"], left_wall_df_interpolate["y"])
nusselt_number = np.abs(nusselt_number)
nusselt_number

In [ ]:
plt.figure(figsize=(8, 6), facecolor="white")
plt.plot(left_wall_df["y"], left_wall_df["dtdx"], "o", label="Original", zorder=10, color="r")
plt.plot(left_wall_df_interpolate["y"], left_wall_df_interpolate["dtdx"], "-", label="Interpolate", color="blue")
plt.xlabel("$y$", fontsize=15)
plt.ylabel("$\\frac{dT}{dx}$", fontsize=15)
plt.title("Left Wall $\\frac{dT}{dx}$, $Nu=$%s" % str(np.round(nusselt_number, 5)), fontsize=15)
plt.legend()
plt.grid()
plt.savefig("images/"+ra_str+"_left_wall_dtdx.pdf", bbox_inches="tight", dpi=300)
plt.savefig("images/"+ra_str+"_left_wall_dtdx.png", bbox_inches="tight", dpi=300)
plt.show()